# rGreat online vs local 

This notebook allows you to compare results between local and online rGreat. 

r-base == 3.6.1

r-base required for rGreat is 3.6.0

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import pandas as pd

In [3]:
rpy2.__version__

'3.5.2'

In [3]:
importr('rGREAT')

rpy2.robjects.packages.Package as a <module 'rGREAT'>

In [12]:
res_online = rpy2.robjects.r['submitGreatJob']("../../data/tests/test_data/input/07_height_snps_hg19.bed")
res_online = rpy2.robjects.r['getEnrichmentTables'](res_online)

R[write to console]: Note: On Aug 19 2019 GREAT released version 4 which supports hg38
genome and removes some ontologies such pathways. submitGreatJob()
still takes hg19 as default. hg38 can be specified by argument `species
= "hg38"`. To use the older versions such as 3.0.0, specify as
submitGreatJob(..., version = "3"). Set argument `help` to `FALSE` to
turn off this message.

R[write to console]: The default enrichment tables contain no associated genes for the input
regions. You can set `download_by = 'tsv'` to download the complete
table, but note only the top 500 regions can be retreived. See the
following link:

https://great-help.atlassian.net/wiki/spaces/GREAT/pages/655401/Export#Export-GlobalExport



In [4]:
res_local = rpy2.robjects.r['great']("../../data/tests/test_data/input/07_height_snps_hg19.bed", "MSigDB:H", "TxDb.Hsapiens.UCSC.hg19.knownGene")
res_local = rpy2.robjects.r['getEnrichmentTables'](res_local)

R[write to console]: * TSS source: TxDb.

R[write to console]: * check whether TxDb package 'TxDb.Hsapiens.UCSC.hg19.knownGene' is installed.

R[write to console]: * gene ID type in the extended TSS is 'Entrez Gene ID'.

R[write to console]: * restrict chromosomes to 'chr1, chr2, chr3, chr4, chr5, chr6, chr7, chr8, chr9, chr10,
    chr11, chr12, chr13, chr14, chr15, chr16, chr17, chr18, chr19, chr20, chr21, chr22, chrX,
    chrY, chrM'.

R[write to console]: * 18563/23033 protein-coding genes left.

R[write to console]: * update seqinfo to the selected chromosomes.

R[write to console]: * TSS extension mode is 'basalPlusExt'.

R[write to console]: * construct the basal domains by extending 5000bp to upstream and 1000bp to downsteram of TSS.

R[write to console]: * calculate distances to neighbour regions.

R[write to console]: * extend to both sides until reaching the neighbour genes or to the maximal extension (1000000bp).

R[write to console]: Error in h(simpleError(msg, call)) : 
  

RRuntimeError: Error in h(simpleError(msg, call)) : 
  error in evaluating the argument 'INDEX' in selecting a method for function 'tapply': unable to find an inherited method for function ‘seqnames’ for signature ‘"character"’


In [32]:
print(res_online.names)

[1] "GO Molecular Function" "GO Biological Process" "GO Cellular Component"



In [30]:
print(res_online.rx2("GO Molecular Function").colnames)

 [1] "ID"                         "name"                      
 [3] "Binom_Genome_Fraction"      "Binom_Expected"            
 [5] "Binom_Observed_Region_Hits" "Binom_Fold_Enrichment"     
 [7] "Binom_Region_Set_Coverage"  "Binom_Raw_PValue"          
 [9] "Binom_Adjp_BH"              "Hyper_Total_Genes"         
[11] "Hyper_Expected"             "Hyper_Observed_Gene_Hits"  
[13] "Hyper_Fold_Enrichment"      "Hyper_Gene_Set_Coverage"   
[15] "Hyper_Term_Gene_Coverage"   "Hyper_Raw_PValue"          
[17] "Hyper_Adjp_BH"             



In [33]:
online = pd.DataFrame({
    "id":list(res_online.rx2("GO Molecular Function").rx2("ID"))+list(res_online.rx2("GO Biological Process").rx2("ID"))+list(res_online.rx2("GO Cellular Component").rx2("ID")),
    "term":list(res_online.rx2("GO Molecular Function").rx2("name"))+list(res_online.rx2("GO Biological Process").rx2("name"))+list(res_online.rx2("GO Cellular Component").rx2("name")),
    "binom_p_val" : list(res_online.rx2("GO Molecular Function").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Binom_Raw_PValue")),
    "hyper_p_val":list(res_online.rx2("GO Molecular Function").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Hyper_Raw_PValue"))
    })
online.sort_values(by=["binom_p_val"])

,id,term,binom_p_val,hyper_p_val
4219,GO:2000772,regulation of cellular senescence,2.873591e-21,0.001120
4220,GO:0090342,regulation of cell aging,8.617844e-21,0.002480
4221,GO:0090402,oncogene-induced cell senescence,1.727243e-18,0.000239
4222,GO:0035986,senescence-associated heterochromatin focus as...,1.203867e-17,0.000708
17364,GO:0035985,senescence-associated heterochromatin focus,1.203867e-17,0.000708
...,...,...,...,...
9349,GO:0008090,retrograde axonal transport,1.000000e+00,1.000000
9350,GO:0008105,asymmetric protein localization,1.000000e+00,1.000000
9351,GO:0008156,negative regulation of DNA replication,1.000000e+00,1.000000
9353,GO:0008207,C21-steroid hormone metabolic process,1.000000e+00,1.000000


In [39]:
import os 
import matplotlib.pyplot as plt
import greatpy as great 
from scipy.stats import pearsonr

[autoreload of greatpy.pl failed: Traceback (most recent call last):
  File "/home/tom/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/tom/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/tom/miniconda3/envs/gpy/lib/python3.8/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'greatpy' not in sys.modules
]
[autoreload of greatpy.pp failed: Traceback (most recent call last):
  File "/home/tom/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/tom/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/tom/miniconda3/envs/gpy/lib/python3.8/importlib/__init__.py", line 1

ModuleNotFoundError: No module named 'yaml'

In [ ]:
def test(): 
    stat_df = {
        "name" : [],
        "pearson_binom": [], 
        "pearson_hypergeom": []
    }

    for name in os.listdir("../../data/tests/test_data/input"): 
        # online test 
        res_online = rpy2.robjects.r['submitGreatJob'](f"../../data/tests/test_data/input/{name}")
        res_online = rpy2.robjects.r['getEnrichmentTables'](res_online)

        # local test
        res_local = rpy2.robjects.r['great'](f"../../data/tests/test_data/input/{name}", "MSigDB:H", "TxDb.Hsapiens.UCSC.hg19.knownGene")
        res_local = rpy2.robjects.r['getEnrichmentTables'](res_local)

        # create dataframe
        online = pd.DataFrame({
            "id":list(res_online.rx2("GO Molecular Function").rx2("ID"))+list(res_online.rx2("GO Biological Process").rx2("ID"))+list(res_online.rx2("GO Cellular Component").rx2("ID")),
            "term":list(res_online.rx2("GO Molecular Function").rx2("name"))+list(res_online.rx2("GO Biological Process").rx2("name"))+list(res_online.rx2("GO Cellular Component").rx2("name")),
            "binom_p_val" : list(res_online.rx2("GO Molecular Function").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Binom_Raw_PValue")),
            "hyper_p_val":list(res_online.rx2("GO Molecular Function").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Hyper_Raw_PValue"))
            })
        online.sort_values(by=["id"])

        local = pd.DataFrame({
            "id":list(res_local.rx2("GO Molecular Function").rx2("ID"))+list(res_local.rx2("GO Biological Process").rx2("ID"))+list(res_local.rx2("GO Cellular Component").rx2("ID")),
            "term":list(res_local.rx2("GO Molecular Function").rx2("name"))+list(res_local.rx2("GO Biological Process").rx2("name"))+list(res_local.rx2("GO Cellular Component").rx2("name")),
            "binom_p_val" : list(res_local.rx2("GO Molecular Function").rx2("Binom_Raw_PValue"))+list(res_local.rx2("GO Biological Process").rx2("Binom_Raw_PValue"))+list(res_local.rx2("GO Cellular Component").rx2("Binom_Raw_PValue")),
            "hyper_p_val":list(res_local.rx2("GO Molecular Function").rx2("Hyper_Raw_PValue"))+list(res_local.rx2("GO Biological Process").rx2("Hyper_Raw_PValue"))+list(res_local.rx2("GO Cellular Component").rx2("Hyper_Raw_PValue"))
            })
        local.sort_values(by=["id"])

        # compare
        binom_local = []
        hyper_local = []
        binom_online = []
        hyper_online = []
        for i in range(online.shape[0]) : 
            go_id = list(online.index)[i]
            curr_enrichment = online.iloc[i]
            curr_great_webserver = local.loc[local["id"]==go_id]
            binom_local.append(float(curr_enrichment["binom_p_val"]))
            hyper_local.append(float(curr_enrichment["hypergeom_p_val"]))
            binom_online.append(float(curr_great_webserver["binom_p_val"]))
            hyper_online.append(float(curr_great_webserver["hyper_p_val"]))
        binom = pd.DataFrame({"binom_local":binom_local,"binom_online":binom_online})
        hyper = pd.DataFrame({"hyper_greatpy":hyper_local,"hyper_online":hyper_online})


        fig = plt.figure(figsize=(10,5),dpi=80)
        fig.subplots_adjust(hspace=0.4, wspace=0.4)
        ax = fig.add_subplot(2, 2, 1)
        great.pl.scatterplot(binom,colname_x="binom_greatpy",colname_y="binom_great",title=None,ax = ax)
        ax = fig.add_subplot(2, 2, 2)
        great.pl.scatterplot(hyper,colname_x="hyper_greatpy",colname_y="hyper_great",title=None,ax = ax)
        fig.suptitle(f"results for {name}")
        plt.show()
        # great.pl.scatterplot(binom,colname_x="binom_greatpy",colname_y="binom_great",title=f"binom comparison for {name}")
        # great.pl.scatterplot(hyper,colname_x="hyper_greatpy",colname_y="hyper_great",title=f"hypergeom comparison for {name}")

        stat_df["name"].append(name)
        stat_df["pearson_binom"].append(pearsonr(binom_great,binom_greatpy)[0])
        stat_df["pearson_hypergeom"].append(pearsonr(hyper_great,hyper_greatpy)[0])
test()